# Import

In [12]:
import tensorflow as tf
import argparse

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Definition of the Architecture

In [13]:
# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 1000
n_encoder_h_2 = 500
n_encoder_h_3 = 250

# 3 hidden layers for decoder
n_decoder_h_1 = 250
n_decoder_h_2 = 500
n_decoder_h_3 = 1000

# Parameters
learning_rate = 0.01
training_epochs = 200
batch_size = 100
display_step = 1

# Batch Normalization 

In [14]:
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

# Definition of the Layer 

In [15]:
def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))

# Definition of the Encoder Part

In [16]:
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [784, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_encoder_h_2, n_encoder_h_3], [n_encoder_h_3], phase_train)

        with tf.variable_scope("code"):
            output = layer(h_3, [n_encoder_h_3, n_code], [n_code], phase_train)

    return output

# Definition of the Decoder Part

In [17]:
def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Variablle, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_decoder_h_2, n_decoder_h_3], [n_decoder_h_3], phase_train)

        with tf.variable_scope("output"):
            output = layer(h_3, [n_decoder_h_3, 784], [784], phase_train)

    return output

# Definition of the Loss

In [18]:
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op

# Training Function

In [19]:
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

# Evaluation Function

In [20]:
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        in_image_op = image_summary("input_image", x)
        
        out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        return val_loss, in_image_op, out_image_op, val_summary_op

# Image Summary

In [21]:
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 28, 28, 1])
    return tf.summary.image(label, tensor_reshaped)

# Main Function

In [22]:
if __name__ == '__main__':

    #if a python file, please use the 4 lines bellow and comment the "n_code = '1'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '2'
    
    #feel free to change with your own 
    log_files_path = 'C:/Users/Ali/logs/'

    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 784]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            saver = tf.train.Saver(max_to_keep=200)

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            train_writer = tf.summary.FileWriter(log_files_path + 'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            val_writer   = tf.summary.FileWriter(log_files_path + 'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)

            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                total_batch = int(mnist.train.num_examples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    train_writer.add_summary(train_summary, sess.run(global_step))

                    validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: mnist.validation.images, phase_train: False})
                    val_writer.add_summary(in_image, sess.run(global_step))
                    val_writer.add_summary(out_image, sess.run(global_step))
                    val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)

                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    saver.save(sess, log_files_path + 'mnist_autoencoder_hidden_' + n_code + '_logs/model-checkpoint-' + '%04d' % (epoch+1), global_step=global_step)


            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: mnist.test.images, phase_train: False})

            print("Test Loss:", test_loss)

Epoch: 0001 cost = 11.564421090
Validation Loss: 11.725205
Epoch: 0002 cost = 9.381576670
Validation Loss: 8.61294
Epoch: 0003 cost = 8.390918994
Validation Loss: 7.376416
Epoch: 0004 cost = 7.775207559
Validation Loss: 7.260479
Epoch: 0005 cost = 7.432604048
Validation Loss: 7.629019
Epoch: 0006 cost = 7.104031122
Validation Loss: 6.881897
Epoch: 0007 cost = 6.865076444
Validation Loss: 6.4956985
Epoch: 0008 cost = 6.711924877
Validation Loss: 6.477389
Epoch: 0009 cost = 6.553516642
Validation Loss: 6.2858257
Epoch: 0010 cost = 6.420095656
Validation Loss: 6.2364507
Epoch: 0011 cost = 6.274219853
Validation Loss: 6.032448
Epoch: 0012 cost = 6.063214063
Validation Loss: 5.946652
Epoch: 0013 cost = 5.995332215
Validation Loss: 5.9253306
Epoch: 0014 cost = 5.937842406
Validation Loss: 5.8148055
Epoch: 0015 cost = 5.883486397
Validation Loss: 5.7961135
Epoch: 0016 cost = 5.842212829
Validation Loss: 5.7508264
Epoch: 0017 cost = 5.807370794
Validation Loss: 5.6705456
Epoch: 0018 cost = 5.7

Validation Loss: 4.925441
Epoch: 0144 cost = 5.007147345
Validation Loss: 4.918103
Epoch: 0145 cost = 5.001471501
Validation Loss: 4.9182243
Epoch: 0146 cost = 5.001704925
Validation Loss: 4.912139
Epoch: 0147 cost = 5.005144435
Validation Loss: 4.9133644
Epoch: 0148 cost = 4.998747677
Validation Loss: 4.927323
Epoch: 0149 cost = 4.994594357
Validation Loss: 4.911391
Epoch: 0150 cost = 4.997181152
Validation Loss: 4.90724
Epoch: 0151 cost = 4.991022023
Validation Loss: 4.935663
Epoch: 0152 cost = 4.996874541
Validation Loss: 4.907487
Epoch: 0153 cost = 4.988664458
Validation Loss: 4.9254766
Epoch: 0154 cost = 4.995408467
Validation Loss: 4.9257865
Epoch: 0155 cost = 4.994174903
Validation Loss: 4.9086046
Epoch: 0156 cost = 4.989940336
Validation Loss: 4.9452076
Epoch: 0157 cost = 4.985709067
Validation Loss: 4.9176507
Epoch: 0158 cost = 4.979076055
Validation Loss: 4.8860145
Epoch: 0159 cost = 4.984639178
Validation Loss: 4.9171424
Epoch: 0160 cost = 4.985282248
Validation Loss: 4.8877